In [ ]:
# %matplotlib inline
# import matplotlib.pyplot as plt
# import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow.keras import backend as k
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding, Dropout, LSTM, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import model_from_json

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

import pickle

# print(tf.__version__)
# print(tf.keras.__version__)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# This code is based on a tutorial which can be found here:
# https://realpython.com/python-keras-text-classification/#what-is-a-word-embedding

In [ ]:
#Packages to import (regular expressions and pandas)

import re 
import pandas as pd 

from pathlib import Path
import gzip
import json

# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#Mount google colab to google drive so it can access files
from google.colab import drive
drive.mount('/content/drive')

#The huge gzipped JSON files ('NL_jobs.15.jsonl.gz') are located in this folder
cwd = '/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments'

#Print the content of the folder
!ls '/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments'

# set current working directory, This folder should only contain the huge json files.
%cd '/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments'


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
'Baseline Model (BOW Log. regression).ipynb'
 BERT_alt_deploy.ipynb
'BERT_Jads_Nikita- Clyde edit - hpartuning.ipynb'
 BOW_LSTM.ipynb
'Copy of Copy of BERT_alt _ hyperparamtuning.ipynb'
 Cow
 COW_LSTM_hyperparamtuning.ipynb
 dev.gsheet
 dev.tsv
 df3_updated_clean.txt
 df3_updated.txt
 df4_updated_clean.txt
 df4_updated.txt
 Exp23_RNDSCV.csv
 Exp25_RNDSCV.csv
 Exp26_RNDSCV.csv
 Exp27_RNDSCV.csv
 Exp28_RNDSCV.csv
 Exp29_RNDSCV.csv
 Exp30_RNDSCV.csv
 Exp31_RNDSCV

In [ ]:
TRAIN_FILE = "train.tsv"
DEV_FILE = "dev.tsv"

print("TRAIN FILE: \t\t{}\nDEVELOPMENT FILE: \t{}".format(TRAIN_FILE, DEV_FILE))

TRAIN FILE: 		train.tsv
DEVELOPMENT FILE: 	dev.tsv


In [ ]:
# Load the data
test_full = pd.read_csv(TRAIN_FILE, sep = "\t")
dev_full = pd.read_csv(DEV_FILE, sep = "\t")

# Tirm the data to only one we need
test_trim = test_full[["Quality", "#1 String"]]
dev_trim = dev_full[["Quality", "#1 String"]]

# Seperate our data for training and testing
train_text = np.array(test_full["#1 String"])
train_target = np.array(test_full["Quality"])

test_text = np.array(dev_full["#1 String"])
test_target = np.array(dev_full["Quality"])

# Check our training and testing sets
print("Train-set size: ", len(train_text))
print("Test-set size:  ", len(test_text))

# Stack text for tokenization
data_text = np.hstack((train_text, test_text))
print("Shape of the stacked text: {}".format(data_text.shape))

Train-set size:  2240
Test-set size:   720
Shape of the stacked text: (2960,)


In [ ]:
# Get stop words (or most commoon ones basically) for the dutch language
stop_words = set(stopwords.words('dutch'))

# Get the tokinized pattern of all sentences
word_tokens = []
for sentence in data_text:
    word_tokens.append(nltk.word_tokenize(sentence))

# Initiate a new collector
filtered_sentence = []
for entry in word_tokens:
    filtered_sentence.append([w for w in entry if not w in stop_words and w.isalpha()])
filtered_sentence = np.array(filtered_sentence)

# Implement the top level tokinizer, we will go for 10K words
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(filtered_sentence)

In [ ]:
# tokenizer.word_index # If we need to check it

In [ ]:
# Save the tokinizer for re-usability with the new data
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:
x_train_tokens = tokenizer.texts_to_sequences(train_text)
x_test_tokens = tokenizer.texts_to_sequences(test_text)

vocab_size = len(tokenizer.word_index) + 1  

# Example of conversions
print("Conversion Examples:")
print(x_train_tokens[50])
print(train_text[50])
print("---------------------------------")
print(x_train_tokens[100])
print(train_text[100])
print("---------------------------------")

# Padding the data if needed
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)
print("Average Number of Tokens per sentence: ", np.round(np.mean(num_tokens), 4))
print("Maxmimum Number of Tokens per sentence: ", np.max(num_tokens))
print("Minimum Number of Tokens per sentence: ", np.min(num_tokens))

# The max number of tokens we will allow is set to the average plus 2 standard deviations.
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
print("Maximum amount of tokens to hold: ", max_tokens)

# This covers almost 96% of the data
print("How much data does the padding include fully: {}".format(np.sum(num_tokens < max_tokens) / len(num_tokens)))

# Initiate the condition for padding
pad = 'pre'

# Applying padding to the training data
x_train_pad = pad_sequences(x_train_tokens,
                            maxlen=max_tokens,
                            padding=pad,
                            truncating=pad)

# Applying padding to the development data

x_test_pad = pad_sequences(x_test_tokens, 
                           maxlen=max_tokens, 
                           padding=pad, 
                           truncating=pad)

# Verify that all of the padding is done correctly
print("Shape of the padded testing data: \t", x_train_pad.shape)
print("Shape of the padded development data: \t", x_test_pad.shape)
# Seems correct

Conversion Examples:
[233, 2386, 245, 2387]
Utrecht, Drenthe en Flevoland.
---------------------------------
[13, 152, 614, 298, 1211, 2416, 298, 2417, 462, 237, 298, 2418, 835, 115, 823, 2419, 710]
het, met behulp van ons calculatieprogramma BV6 van TSD uit Zwolle, uitwerken van prijsaanvragen tot een correcte werkomschrijvende offerte
---------------------------------
Average Number of Tokens per sentence:  12.7966
Maxmimum Number of Tokens per sentence:  58
Minimum Number of Tokens per sentence:  0
Maximum amount of tokens to hold:  29
How much data does the padding include fully: 0.9618243243243243
Shape of the padded testing data: 	 (2240, 29)
Shape of the padded development data: 	 (720, 29)


In [ ]:
def create_model(lstm1, lstm2, dense1, maxlen):
    #Creating the network
    # embedding_size = 256
    # num_words = num_words # We chose this value earlier (10,000)
    # max_tokens = max_tokens # This values is also chosen from before (30)

    # For reproducability setting a seed
    seed = 42
    # np.random.seed(seed)

    model = Sequential()
    model.add(Embedding(input_dim=vocab_size,
                        output_dim=embedding_dim,
                        weights = [embedding_matrix], 
                        trainable = True,
                        input_length=maxlen,
                        name='layer_embedding'))
    model.add(Bidirectional(LSTM(units = lstm1, return_sequences=True, name = "BiLSTM_1")))
    model.add(Dropout(0.5, noise_shape=None, seed=seed))
    model.add(Bidirectional(LSTM(units = lstm2, return_sequences=False, name = "BiLSTM_2")))
    model.add(Dropout(0.5, noise_shape=None, seed=seed))
    model.add(Dense(dense1, activation='relu', name = "Intermediate_1"))
    Dropout(rate = 0.5, noise_shape=None, seed=seed)
    # model.add(Dense(16, activation='relu', name = "Intermediate_2"))
    model.add(Dense(1, activation='sigmoid', name = "Final_Output"))
    optimizer = Adam(lr = 1e-3)
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['acc'])
    # model.summary()

    return model

In [ ]:
param_grid = dict(lstm1=[4,8,16, 32],
                  lstm2=[4, 8,16, 32],
                  dense1=[4, 8,16, 32],          
                  # vocab_size=[5000], 
                  # embedding_dim=[50, 30, 80],
                  maxlen=[29,50])

# param_grid = dict(lstm1=[16, 32],
#                   lstm2=[16, 32],
#                   dense1=[16, 32],          
#                   vocab_size=[5000], 
#                   embedding_dim=[50],
#                   maxlen=[29],
#                    random_state=1, refit=True,
#                    return_train_score=True
#                   )


# param_grid = dict(batch_size_param=[16, 32, 64, 128],
#                   lr_param=[1e-3, 5e-5, 3e-5, 2e-5],          
#                   vocab_size=[5000], 
#                   embedding_dim=[50],
#                   maxlen=[29],       
#                    )


# param_grid = dict(num_filters=[32, 64, 128],
#                   kernel_size=[3, 5, 7],
#                   vocab_size=[vocab_size],
#                   embedding_dim=[embedding_dim],
#                   maxlen=[maxlen])

In [ ]:
# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 5)
# model = KerasClassifier(build_fn=create_model,
#                         epochs=3, batch_size= 64,
#                         verbose=False,
#                         validation_data=(X_test, test_target))
#                         ,
#                         callbacks=[es])

In [ ]:
import numpy as np

def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath, encoding="latin-1") as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

In [ ]:
embedding_dim = 50
# str = unicode(str, errors='ignore')
embedding_matrix = create_embedding_matrix(
    'NL_Embedding/model.txt',   
    tokenizer.word_index, embedding_dim)

# with open(path, 'rb') as f:
#   contents = f.read()


In [ ]:
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
nonzero_elements / vocab_size


0.9412027721886876

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV



output_file = 'data/output.txt'


from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import fbeta_score, make_scorer

epochs = 15 
cv_folds = 4
n_iter = 40
batch_size = 128

model = KerasClassifier(build_fn=create_model,
                        epochs=epochs, batch_size= batch_size,
                        verbose=False
                       )

grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
                          cv= cv_folds, verbose=1, n_iter= n_iter,
                          # scoring= score,
                          return_train_score = True,
                          n_jobs = -1,
                          refit = True
                          )





from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping
# early_stopping = EarlyStopping(monitor='loss', patience=5)


grid_result = grid.fit(x_train_pad, train_target
                      #  , callbacks=[early_stopping]
                       )

# Evaluate testing set
test_accuracy = grid.score(x_test_pad, test_target)


print(grid_result.best_score_)
print(grid_result.best_params_)
# print(test_accuracy)
print(grid_result)

Fitting 4 folds for each of 40 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 13.3min
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed: 45.6min finished


0.8218749910593033
{'maxlen': 29, 'lstm2': 4, 'lstm1': 32, 'dense1': 16}
RandomizedSearchCV(cv=4, error_score=nan,
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f8abeadb908>,
                   iid='deprecated', n_iter=40, n_jobs=-1,
                   param_distributions={'dense1': [4, 8, 16, 32],
                                        'lstm1': [4, 8, 16, 32],
                                        'lstm2': [4, 8, 16, 32],
                                        'maxlen': [29, 50]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=True, scoring=None, verbose=1)


In [ ]:
df_end = pd.DataFrame(grid_result.cv_results_)
df_end
# df_end.to_csv('Exp35_RNDSCV.csv')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_maxlen,param_lstm2,param_lstm1,param_dense1,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,mean_train_score,std_train_score
0,33.751526,0.181117,2.276141,0.058858,29,4,32,16,"{'maxlen': 29, 'lstm2': 4, 'lstm1': 32, 'dense...",0.808929,0.841071,0.860714,0.776786,0.821875,0.031928,1,0.967857,0.936905,0.948810,0.963095,0.954167,0.012184
1,40.178628,0.271007,2.478999,0.161091,29,32,32,32,"{'maxlen': 29, 'lstm2': 32, 'lstm1': 32, 'dens...",0.808929,0.835714,0.835714,0.775000,0.813839,0.024948,4,0.985714,0.981548,0.983333,0.970833,0.980357,0.005694
2,26.121745,0.243416,3.059890,0.752174,50,8,8,8,"{'maxlen': 50, 'lstm2': 8, 'lstm1': 8, 'dense1...",0.816071,0.857143,0.798214,0.758929,0.807589,0.035296,18,0.972619,0.969643,0.974405,0.972619,0.972321,0.001710
3,39.136079,0.455879,2.455562,0.150224,50,32,32,16,"{'maxlen': 50, 'lstm2': 32, 'lstm1': 32, 'dens...",0.769643,0.848214,0.800000,0.778571,0.799107,0.030423,28,0.985714,0.985714,0.920238,0.982738,0.968601,0.027949
4,25.588007,0.506728,2.684751,0.482199,29,8,8,4,"{'maxlen': 29, 'lstm2': 8, 'lstm1': 8, 'dense1...",0.823214,0.850000,0.837500,0.767857,0.819643,0.031365,2,0.979762,0.952976,0.960714,0.960714,0.963542,0.009883
5,28.454277,0.355611,2.414179,0.224679,29,4,16,8,"{'maxlen': 29, 'lstm2': 4, 'lstm1': 16, 'dense...",0.771429,0.862500,0.837500,0.766071,0.809375,0.041619,14,0.942262,0.926190,0.949405,0.963690,0.945387,0.013504
6,33.741036,0.185806,2.401493,0.174686,50,8,32,8,"{'maxlen': 50, 'lstm2': 8, 'lstm1': 32, 'dense...",0.796429,0.821429,0.848214,0.782143,0.812054,0.025171,7,0.972619,0.966071,0.963690,0.969643,0.968006,0.003403
7,26.755689,0.312345,2.437918,0.197786,50,8,8,4,"{'maxlen': 50, 'lstm2': 8, 'lstm1': 8, 'dense1...",0.810714,0.860714,0.789286,0.780357,0.810268,0.031145,13,0.936310,0.966667,0.961905,0.973214,0.959524,0.013991
8,25.524957,0.485561,2.256302,0.179424,29,8,8,8,"{'maxlen': 29, 'lstm2': 8, 'lstm1': 8, 'dense1...",0.791071,0.819643,0.791071,0.794643,0.799107,0.011946,28,0.971429,0.964881,0.965476,0.960714,0.965625,0.003820
9,28.068346,0.292678,2.306400,0.216266,50,8,16,32,"{'maxlen': 50, 'lstm2': 8, 'lstm1': 16, 'dense...",0.796429,0.828571,0.805357,0.800000,0.807589,0.012524,18,0.979762,0.974405,0.977381,0.976190,0.976935,0.001946


In [ ]:
print(grid_result.best_score_)
print(grid_result.best_params_)

# test_accuracy = grid.score(x_test_pad, test_target)
# print(test_accuracy)
print(grid_result)


# train_accuracy = grid.score(x_train_pad, train_target)
# print(train_accuracy)

0.8218749910593033
{'maxlen': 29, 'lstm2': 4, 'lstm1': 32, 'dense1': 16}
RandomizedSearchCV(cv=4, error_score=nan,
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f8abeadb908>,
                   iid='deprecated', n_iter=40, n_jobs=-1,
                   param_distributions={'dense1': [4, 8, 16, 32],
                                        'lstm1': [4, 8, 16, 32],
                                        'lstm2': [4, 8, 16, 32],
                                        'maxlen': [29, 50]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=True, scoring=None, verbose=1)


In [ ]:
threshold = 0.5
predicted = grid.predict(x_train_pad)

predicted = np.where(predicted > threshold, 1,0)


predicted  = np.reshape(predicted, len(predicted))


eval_predicted = grid.predict(x_test_pad)

eval_predicted = np.where(eval_predicted > threshold, 1,0)

eval_predicted  = np.reshape(eval_predicted, len(eval_predicted))

# train acc

no_predictions = len(predicted)
print(no_predictions)

right = np.sum(predicted == train_target)

training_accuracy = right/ no_predictions
print("Training Accuracy: ", training_accuracy)


# test acc

no_predictions_eval = len(eval_predicted)
print(no_predictions_eval)

right_eval = np.sum(eval_predicted == test_target)

test_accuracy = right_eval/ no_predictions_eval
print("Test Accuracy: ", test_accuracy)

2240
Training Accuracy:  0.9700892857142858
720
Test Accuracy:  0.7597222222222222


In [ ]:
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as sk

# loss, accuracy = grid.score(x_train_pad, train_target)
# print("Training Accuracy: {:.4f}".format(accuracy))
# print("Training Loss: {:.4f}".format(loss))


precision = sk.precision_score(train_target, predicted)
print("Training Precision: ", precision)
recall = sk.recall_score(train_target, predicted)
print("Training recall: ", recall)
roc_auc = sk.roc_auc_score(train_target, predicted)
print("Training roc_auc: ", roc_auc)
f1 = sk.f1_score(train_target, predicted)
print("Training f1: ", f1)


# loss, accuracy = grid.score(x_test_pad, test_target)
# print("Eval Accuracy: {:.4f}".format(accuracy))
# print("Eval Loss: {:.4f}".format(loss))


precision = sk.precision_score(test_target, eval_predicted)
print("Eval Precision: ", precision)
recall = sk.recall_score(test_target, eval_predicted)
print("Eval recall: ", recall)
roc_auc = sk.roc_auc_score(test_target, eval_predicted)
print("Eval roc_auc: ", roc_auc)
f1 = sk.f1_score(test_target, eval_predicted)
print("Eval f1: ", f1)

Training Precision:  0.9506292352371732
Training recall:  0.9839679358717435
Training roc_auc:  0.9714525669696882
Training f1:  0.967011324470704
Eval Precision:  0.6340782122905028
Eval recall:  0.8438661710037175
Eval roc_auc:  0.7767002695373354
Eval f1:  0.7240829346092504
